In [1]:
import json
import re

In [138]:
def extractQueriesData (file):
    inputFile = open(file)
 
    data = json.load(inputFile)

    queriesData = []
 
    print(len(data))
    
    for i in data:
        queriesDict = {}
        for l in i["sql"]:
            queriesDict["query"] = l
            queriesDict["keywords"] = extractKeywords(l)
        queriesData.append(queriesDict)
 
    inputFile.close()
    return queriesData

In [134]:
def extractKeywords (sqlQuery):
    #### any word after . and any before as 
    keywords = set()
    findAttr = re.compile('\.\w+')
    #attributes.extend(findAttr.findall(sqlQuery))
    tempAttr = findAttr.findall(sqlQuery)
    findEntities = re.compile('\w+ AS')
    #entities.extend(findEntities.findall(sqlQuery))
    tempEntities = findEntities.findall(sqlQuery)
    for i in tempAttr:
        keywords.add(i.split(".")[1])
    for i in tempEntities:
        keywords.add(i.split(" AS")[0])
    
    ###TO Do: we need to specify type of each keyword
    return list(keywords)

In [140]:
########### main function ###############
queriesData = extractQueriesData("./jsonFiles/scholar.json")
outputFile = open("./outputs/scholar.json", "w")
json.dump(queriesData, outputFile)
outputFile.close()

193


In [14]:
data = "SELECT avg ( salary, gender )  FROM instructor (Nihal) ssgsgsg (   k,   jjj, kks )"

test = re.findall('\(([^)]+)', data)
print(test)

[' salary, gender ', 'Nihal', '   k,   jjj, kks ']


In [76]:

data = "SELECT sum ( balance, T.kdkd ), hdhdhddh , T1.name, (SELECT COUNT (*), NANA FROM HSHS ) FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"O'mahony\" union SELECT sum ( T2.balance ) , T1.name FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"Wang\" union SELECT sum ( T2.balance ) , T1.name FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"Brown\""

keywords = []

selectPart = re.findall('SELECT (.*?) FROM', data)

for i in selectPart:
    i = re.sub('\(SELECT',"",i)
    aggrAttr = re.findall('\(([^)]+)', i)
    for attr in aggrAttr:
        tempList = set()
        tempAttr = attr.split(",") #array of attributes
        for i in tempAttr:
            dot = i.find(".")
            if dot == -1:
                tempList.add((i.strip(), "Aggregation"))
            else:
                tempList.add((i.strip().split(".")[1], "Aggregation"))
        keywords.extend(tempList)
print(list(set(keywords)))    

[('*', 'Aggregation'), ('balance', 'Aggregation'), ('kdkd', 'Aggregation')]


In [79]:
### extract select attributes
sql = "SELECT sum ( balance, T.kdkd ), hdhdhddh , T1.name, (SELECT COUNT (*), NANA FROM HSHS ) FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"O'mahony\" union SELECT sum ( T2.balance ) , T1.name FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"Wang\" union SELECT sum ( T2.balance ) , T1.name FROM accounts AS T1 JOIN savings AS T2 ON T1.custid   =  T2.custid  where T1.name  =  \"Brown\""
sql = re.sub('\(([^)]+)',"",sql)
aggrAttr = re.findall('SELECT (.*?) FROM', sql)
keys = []
for l in aggrAttr:
    arr = l.split(",")
    for i in arr:
        if i.find(")") == -1:
            if i.find(".") == -1:
                keys.append((i.strip(),"Select"))
            else:
                keys.append(((i.strip().split(".")[1], "Select")))
print(keys)


[('hdhdhddh', 'Select'), ('name', 'Select'), ('NANA', 'Select'), ('name', 'Select'), ('name', 'Select')]


In [ ]:
########################### Final Code starting from here #########################################

In [54]:
def separate_queries(query):
    queries=[]
    runing_query=[]
    i = 1
    runing_query.append(query[0])
    while i < len(query):
        token = query[i]
        if token == 'as':
            i+=2
            continue 
        if token == 'select':
            prev_token = query[i-1]
            nested_query = []
            if prev_token == '(':
                #WHILE TO GET BRACKET END
                open_bracket = 1
                while open_bracket >=1:
                    token = query[i]
                    i+=1
                    if token == '(': open_bracket += 1
                    elif token == ')': open_bracket -= 1
                    if open_bracket == 0: break
                    nested_query.append(token)
            else:
                nested_query = query[i:]
                i = len(query)+1
            nested_queries = separate_queries(nested_query)
            queries.extend(nested_queries)
        if i <= len(query): runing_query.append(token)
        i+=1
    queries.append(runing_query)
    return queries

In [67]:
### my getqueries function ###
stop_words = ['in','intersect','except','union','not',"desc","asc","or","and","like"]
AggrFunctions = ["count","sum","min","max","avg"]
tokensToSkip = [",","(",")",".","=","!","by","distinct",">",">=","<","<=",";"]
def get_queries(query,realQuery):
    entities = []
    selectAttrs = []
    joinAttrs = []
    groupByAttrs = []
    orderByAttrs = []
    aggrAttrs = []
    whereAttrs = []
    i = 0
    token = ""
    wordType = ""
    orderFunction = ""
    #isValue = False
    while i < len(query):
        token = query[i]
        if token == "select":
            wordType = "select"
            i += 1
            continue
        if token in stop_words:
            i += 1
            continue
        if i+1 < len(query) and query[i+1] == ".": 
            i += 1
            continue
        if token == "limit":
            i += 2
            continue
        # if token == "=" and wordType == "whereClause":
        #     isValue = True
        #     i += 1
        #     continue
        if token in tokensToSkip:
            if token == ")" and wordType == "aggr":
                wordType = "select"
            i += 1
            continue
        if token == "group":
            wordType = "groupBy"
            i += 2
            continue
        if token == "order":
            wordType = "orderBy"
            if query[i+2] in AggrFunctions:
                orderFunction = query[i+2]
                i += 3
            else: 
                orderFunction = ""
                i += 2
            continue
        if token in AggrFunctions:
            wordType = "aggr"
            i+= 1
            continue
        if token == "from" or token == "join":
            wordType = "entity"
            i += 1
            continue
        if token == "where":
            wordType = "whereClause"
            i += 1
            continue
        if token == "on":
            wordType = "join"
            i += 1
            continue
        if token.find("'") != -1 or token.find('"') != -1:
            #isValue = False
            i += 1
            continue
        if token.find("\"") != -1:
            i += 1
            while query[i].find("\"") == -1:
                i += 1
            i += 1
            #isValue = False
            continue
        # if isValue == True: #then it is number which you have to skip
        #     isValue = False
        #     i += 1
        #     continue
        # check for numbers to skip
        if re.search('[0-9]+', token):
            i += 1
            continue

        if wordType == "select":
            selectAttrs.append(token)
        elif wordType == "entity":
            entities.append(token)
        elif wordType == "join":
            joinAttrs.append(token)
        elif wordType == "groupBy":
            groupByAttrs.append(token)
        elif wordType == "orderBy":
            orderByAttrs.append((token, orderFunction))
        elif wordType == "aggr":
            aggrAttrs.append(token)
        elif wordType == "whereClause":
            whereAttrs.append(token)
        i += 1
    tempDict = {}
    tempDict["query"] = realQuery
    tempDict["entities"] = list(set(entities))
    tempDict["selectAttrs"] = list(set(selectAttrs))
    tempDict["joinAttrs"] = list(set(joinAttrs))
    tempDict["groupByAttrs"] = list(set(groupByAttrs))
    tempDict["orderByAttrs"] = list(set(orderByAttrs))
    tempDict["aggrAttrs"] = list(set(aggrAttrs))
    tempDict["whereAttrs"] = list(set(whereAttrs))
    return tempDict

In [24]:
### for testing ###
data = [
            "select",
            "name",
            "from",
            "track",
            "where",
            "milliseconds",
            "=",
            "(",
            "select",
            "max",
            "(",
            "milliseconds",
            ")",
            "from",
            "track",
            ")"
        ]

queries = separate_queries(data)
dataset = []
for query in queries:
    tempDict = get_queries(query)
    dataset.append(tempDict)
print(dataset)

[{'entities': ['track'], 'selectAttrs': [], 'joinAttrs': [], 'groupByAttrs': [], 'orderByAttrs': [], 'aggrAttrs': ['milliseconds'], 'whereAttrs': []}, {'entities': ['track'], 'selectAttrs': ['name'], 'joinAttrs': [], 'groupByAttrs': [], 'orderByAttrs': [], 'aggrAttrs': [], 'whereAttrs': ['milliseconds']}]


In [65]:
### for testing ###
query_tokens =  [
            "select",
            "t1",
            ".",
            "name",
            "from",
            "genres",
            "as",
            "t1",
            "join",
            "tracks",
            "as",
            "t2",
            "on",
            "t1",
            ".",
            "id",
            "=",
            "t2",
            ".",
            "genre_id",
            "group",
            "by",
            "t1",
            ".",
            "name"
        ]

queries = []
current_queries = get_queries(query_tokens)      
queries.extend(current_queries)
print(queries)

[('genre_id', 'join'), ('name', 'groupBy'), ('tracks', 'entity'), ('genres', 'entity'), ('id', 'join'), ('name', 'select')]


In [78]:
#### update separate_queries to be suitable for sparc dataset ####
def separate_queries_sparc(query):
    queries=[]
    runing_query=[]
    i = 1
    runing_query.append(query[0])
    while i < len(query):
        token = query[i]
        if token == 'select':
            prev_token = query[i-1]
            nested_query = []
            if prev_token == '(':
                #WHILE TO GET BRACKET END
                open_bracket = 1
                while open_bracket >=1:
                    token = query[i]
                    i+=1
                    if token == '(': open_bracket += 1
                    elif token == ')': open_bracket -= 1
                    if open_bracket == 0: break
                    nested_query.append(token)
            else:
                nested_query = query[i:]
                i = len(query)+1
            nested_queries = separate_queries(nested_query)
            queries.extend(nested_queries)
        if i <= len(query): runing_query.append(token)
        i+=1
    queries.append(runing_query)
    return queries

In [82]:
#### update get_queries to be suitable for sparc dataset ####
stop_words = ['in','intersect','except','union','not',"desc","asc","or","and","like"]
AggrFunctions = ["count","sum","min","max","avg"]
tokensToSkip = [",","(",")",".","=","!","by","distinct",">",">=","<","<=",";"]
def get_queries_sparc(query,realQuery):
    entities = []
    selectAttrs = []
    joinAttrs = []
    groupByAttrs = []
    orderByAttrs = []
    aggrAttrs = []
    whereAttrs = []
    i = 0
    token = ""
    wordType = ""
    orderFunction = ""
    while i < len(query):
        token = query[i]
        if token == "select":
            wordType = "select"
            i += 1
            continue
        if token in stop_words:
            i += 1
            continue
        if i+1 < len(query) and query[i+1] == ".": 
            i += 1
            continue
        if token == "limit":
            i += 2
            continue
        stringSeparators = ["\"","'",'"']
        if token in stringSeparators:
            i += 1
            while query[i] not in stringSeparators:
                i += 1
            i += 1
            continue
        if token in tokensToSkip:
            if token == ")" and wordType == "aggr":
                wordType = "select"
            i += 1
            continue
        if token == 'as':
            i+=2
            continue 
        if token == "group":
            wordType = "groupBy"
            i += 2
            continue
        if token == "order":
            wordType = "orderBy"
            if query[i+2] in AggrFunctions:
                orderFunction = query[i+2]
                i += 3
            else: 
                orderFunction = ""
                i += 2
            continue
        if token in AggrFunctions:
            wordType = "aggr"
            i+= 1
            continue
        if token == "from" or token == "join":
            wordType = "entity"
            i += 1
            continue
        if token == "where":
            wordType = "whereClause"
            i += 1
            continue
        if token == "on":
            wordType = "join"
            i += 1
            continue
        # check for numbers to skip
        if re.search('[0-9]+', token):
            i += 1
            continue

        if wordType == "select":
            selectAttrs.append(token)
        elif wordType == "entity":
            entities.append(token)
        elif wordType == "join":
            joinAttrs.append(token)
        elif wordType == "groupBy":
            groupByAttrs.append(token)
        elif wordType == "orderBy":
            orderByAttrs.append((token, orderFunction))
        elif wordType == "aggr":
            aggrAttrs.append(token)
        elif wordType == "whereClause":
            whereAttrs.append(token)
        i += 1
    tempDict = {}
    tempDict["query"] = realQuery
    tempDict["entities"] = list(set(entities))
    tempDict["selectAttrs"] = list(set(selectAttrs))
    tempDict["joinAttrs"] = list(set(joinAttrs))
    tempDict["groupByAttrs"] = list(set(groupByAttrs))
    tempDict["orderByAttrs"] = list(set(orderByAttrs))
    tempDict["aggrAttrs"] = list(set(aggrAttrs))
    tempDict["whereAttrs"] = list(set(whereAttrs))
    return tempDict

In [95]:
############# this cell for datasets:  sparc and splash-master and cosql ###########
import json
import re
keywordsDict = []
with open('/home/nihal/Desktop/GP/dataset_preprocessing/splash-master/train.json', 'r',encoding='UTF-8') as file:
    data = json.load(file)
    for query in data:
        nestedQueries = []
        query_tokens = query['query_toks']
        separatedQueries = separate_queries_sparc(query_tokens)  
        for currentQuery in separatedQueries:
            concatQuery = ' '.join(currentQuery)
            concatQuery = concatQuery.replace(" . ",".")
            tempDict = get_queries_sparc(currentQuery,concatQuery)
            nestedQueries.append(tempDict)
        keywordsDict.append(nestedQueries)

outputFile = open("./outputs/splash-master_train.json", "w")
json.dump(keywordsDict, outputFile)
outputFile.close()

In [94]:
################ testing joins for query tokens ###################
query_toks = ["select", "count", "(", "*", ")", "from", "student", "where", "sex", "=", "\"", "m", "\"", "and", "stuid", "in", "(", "select", "stuid", "from", "has_allergy", "as", "t1", "join", "allergy_type", "as", "t2", "on", "t1", ".", "allergy", "=", "t2", ".", "allergy", "where", "t2", ".", "allergytype", "=", "\"", "food", "\"", ")"]
result = ' '.join(query_toks)
result = result.replace(" . ",".")
print(result)

select count ( * ) from student where sex = " m " and stuid in ( select stuid from has_allergy as t1 join allergy_type as t2 on t1.allergy = t2.allergy where t2.allergytype = " food " )


In [87]:
#### splash-master data tokenization #####
import re
import json
sparcDataset = []
with open ("/home/nihal/Desktop/GP/dataset/Splash-master/data/dev.json", 'r',encoding='UTF-8') as file:
    data = json.load(file)
    for query in data:
        tempDict = {}
        tempDict["query"] = query["gold_parse"]
        tokensList = re.split('(\W)', query["gold_parse"])
        tokensList = list(filter(None, tokensList))
        tokensList = list(filter(lambda x: x != ' ', tokensList))
        convertedTokensList = (map(lambda x: x.lower(), tokensList))
        tempDict["query_toks"] = list(convertedTokensList)
        sparcDataset.append(tempDict)

outputFile = open("/home/nihal/Desktop/GP/dataset_preprocessing/splash-master/dev.json", "w")
json.dump(sparcDataset, outputFile)
outputFile.close()

In [80]:
###############test 
test = {"query": "SELECT count(*) FROM DEPARTMENT WHERE Division  =  \"AS\"", "query_toks": ["select", "count", "(", "*", ")", "from", "department", "where", "division", "=", "\"", "as", "\""]}

import json
import re
keywordsDict = [] 
separatedQueries = separate_queries_sparc(test['query_toks'])  
for currentQuery in separatedQueries:
    tempDict = get_queries_sparc(currentQuery,test['query'])
    nestedQueries.append(tempDict)
keywordsDict.append(nestedQueries)

print(keywordsDict)


as
SELECT count(*) FROM DEPARTMENT WHERE Division  =  "AS"
[[{'query': 'SELECT CName FROM COURSE WHERE Days  =  "MTW"', 'entities': ['course'], 'selectAttrs': ['cname'], 'joinAttrs': [], 'groupByAttrs': [], 'orderByAttrs': [], 'aggrAttrs': [], 'whereAttrs': ['days']}, {'query': 'SELECT count(*) FROM DEPARTMENT WHERE Division  =  "AS"', 'entities': ['department'], 'selectAttrs': [], 'joinAttrs': [], 'groupByAttrs': [], 'orderByAttrs': [], 'aggrAttrs': ['*'], 'whereAttrs': ['division']}]]


In [52]:
#### sparc data tokenization #####
import re
import json
sparcDataset = []
with open ("/home/nihal/Desktop/GP/dataset/sparc/sparc/dev.json", 'r',encoding='UTF-8') as file:
    data = json.load(file)
    for query in data:
        tempDict = {}
        tempDict["query"] = query["final"]["query"]
        tokensList = re.split('(\W)', query["final"]["query"])
        tokensList = list(filter(None, tokensList))
        tokensList = list(filter(lambda x: x != ' ', tokensList))
        convertedTokensList = (map(lambda x: x.lower(), tokensList))
        tempDict["query_toks"] = list(convertedTokensList)
        sparcDataset.append(tempDict)

outputFile = open("/home/nihal/Desktop/GP/dataset_preprocessing/sparcDataset/dev.json", "w")
json.dump(sparcDataset, outputFile)
outputFile.close()

In [48]:
l = "SELECT name FROM department GROUP BY departmentID ORDER BY count(departmentID) DESC LIMIT 1;"
print(re.split('(\W)',l))
results = re.split('(\W)',l)

print("---------------------------------------------")
results = list(filter(None, results))
results = list(filter(lambda x: x != ' ', results))
print(results)


['SELECT', ' ', 'name', ' ', 'FROM', ' ', 'department', ' ', 'GROUP', ' ', 'BY', ' ', 'departmentID', ' ', 'ORDER', ' ', 'BY', ' ', 'count', '(', 'departmentID', ')', '', ' ', 'DESC', ' ', 'LIMIT', ' ', '1', ';', '']
---------------------------------------------
['SELECT', 'name', 'FROM', 'department', 'GROUP', 'BY', 'departmentID', 'ORDER', 'BY', 'count', '(', 'departmentID', ')', 'DESC', 'LIMIT', '1', ';']
